### Import Libraries

In [ ]:
import pandas as pd 
import psycopg2
from dotenv import load_dotenv
import os
import glob
import openpyxl
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime
import numpy as np

In [ ]:
load_dotenv("credentials.env")
DATABASE = os.getenv('DATABASE')
HOST = os.getenv('HOST')
USER = os.getenv('USER')
PASSWORD = os.getenv('PASSWORD')
PORT = os.getenv('PORT')

### Query from DB

In [ ]:
conn= psycopg2.connect(database=DATABASE,
                       host= HOST,
                       user=USER,
                       password=PASSWORD,
                       port=PORT)

#### Query (change to select date range)

In [ ]:
date = 'sept2023'
startdate = '2023-9-01'
enddate =  '2023-9-30'

cursor = conn.cursor()

query = "Query Inserted Here"

cursor.execute(query,(enddate,startdate,startdate,enddate))


results=cursor.fetchall()
columns = ['Columns Pulled Here']
df = pd.DataFrame(results,columns=columns)

cursor.close()
conn.close()

# Save to a csv for the pulled month
filename = f"PulledMonth/{date}.csv"
df.to_csv(filename,index=False)


### Joined Months into one Dataset

In [ ]:
# Get CSV files list from a folder
path = "PulledMonth"
csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file) for file in csv_files)

# Concatenate all DataFrames
df = pd.concat(df_list, ignore_index=True)

#### Data Cleaning

In [ ]:
df['contract_end_date'] = pd.to_datetime(df.contract_end_date, format='%Y-%m-%d')
df['month'] = df['contract_end_date'].dt.strftime('%m')
df['year'] = df['contract_end_date'].dt.strftime('%Y')

### Cleaning plan name mistakes

In [ ]:
df.drop_duplicates(subset=['account number','End Date of Contract'],inplace=True)
df.next_plan.replace({np.nan: 'None'},inplace=True)
df.plan_name.replace({'Typo in Plan Name': 'Corrected Plan Name'},inplace=True)
df.next_plan.replace({'Typo in Plan Name': 'Corrected Plan Name'},inplace=True)


In [ ]:
df['contract_start_date'] = pd.to_datetime(df['contract start date'])

In [ ]:
df['actual_end_date'] = df['contract start date'] + df['plan term'].apply(lambda x: pd.offsets.DateOffset(months=x))

In [ ]:
df[['contract_start_date','actual_end_date','contract end date']] = df[['contract_start_date','actual_end_date','contract end date']].apply(pd.to_datetime)
df['days_between'] = (df['actual_end_date']-df['contract end date']) / np.timedelta64(1, 'D')

#### Determine if they made it to the end of their Contract

In [ ]:
def categorize(row):
    if row['days_between'] < 7 and row['days_between'] > -60 and row['plan term'] > 6:
        return 1
    elif row['days_between'] < 7 and row['days_between'] > -30 and row['plan term'] <= 6:
        return 1
    return 0

df['made_it_to_contract'] = df.apply(lambda row: categorize(row),axis=1)

In [ ]:
df[['actual_end_date','contract_start_date','contract_end_date','plan_term','days_between','made_it_to_contract','next plan term']]

In [ ]:
df.next_plan_term.fillna(0,inplace=True)

#### Next Term Length

In [ ]:
# did the customer go to a longer term length, shorter, or same term length?
def sametermlength(row):
    if row['plan_term'] == row['next_plan_term']:
        return 1
    return 0
df['same_term_length'] = df.apply(lambda row: sametermlength(row),axis=1)


def shortertermlength(row):
    if row['plan_term'] > row['next_plan_term']:
        return 1
    return 0
df['shoter_term_length'] = df.apply(lambda row: shortertermlength(row),axis=1)

def longertermlength(row):
    if row['plan_term'] < row['next_plan_term']:
        return 1
    return 0
df['longer_term_length'] = df.apply(lambda row: longertermlength(row),axis=1)

def movedtovariable(row):
    if (row['next_plan_term'] == 1 and ( row['next_plan'] != "RollOff, Plan")):
        return 1
    return 0
df['moved_to_variable'] = df.apply(lambda row: movedtovariable(row),axis=1)

# Did the customer renewed, rolled, or leave?
def rolled(row):
    if (row['next_plan'] == "RollOff Plan") :
        return 1
    return 0
df['rolled'] = df.apply(lambda row: rolled(row),axis=1)

def customer_left(row):
    if row['next_plan_term'] == 0 or row['next_plan']=='None' or ['next_plan_term'] == np.nan or row['next_plan'] == '':
        return 1
    return 0
df['customer_left'] = df.apply(lambda row: customer_left(row),axis=1)

def customer_renewed(row):
    if (row['next_plan_term'] > 1 ) :
        return 1
    return 0
df['customer_renewed'] = df.apply(lambda row: customer_renewed(row),axis=1)


# TermLengthChange
def LongerTerm(row):
    if (row['next_plan_term'] > row['plan_term']):
        return 1
    return 0
df['LongerTerm'] = df.apply(lambda row: LongerTerm(row),axis=1)

def ShorterTerm(row):
    if (row['next_plan_term'] < row['plan_term'] and row['next_plan_term'] > 0 ) and (row['next_plan'] != "RollOff, Plan"):
        return 1
    return 0
df['ShorterTerm'] = df.apply(lambda row: ShorterTerm(row),axis=1)

def SameTerm(row):
    if (row['next_plan_term'] == row['plan_term'] ):
        return 1
    return 0
df['SameTerm'] = df.apply(lambda row: SameTerm(row),axis=1)

def toVariable(row):
    if (row['next_plan_term'] == 1 and (row['next_plan']!="RollOff, Plan")):
        return 1
    return 0
df['toVariable'] = df.apply(lambda row: toVariable(row),axis=1)

In [ ]:
df['plan_name'].replace("RollOff, Plan", "RollOff Plan",inplace=True)
# df['plan_name'].replace("Simply Electricity, Y'all", "Simply Electricity Yall",inplace=True)

In [ ]:
df['plan_name'] = np.where((df['plan_name'] != 'RollOff Plan') & (df['plan_term'] == 1), "Variable Plan", df['plan_name'])
df['next_plan'] = np.where((df['next_plan'] != 'RollOff Plan') & (df['next_plan_term'] == 1), "Variable Plan", df['next_plan'])

#### Query Filters for Each Section

In [ ]:
queries = [
    {"query_name":"AllPlans","conditions":(df==df)},
    {"query_name":"FixedPlans","conditions":(df['plan_term'] != 1)},
    {"query_name":"EndofContract","conditions":(df['is_sherpa'] != "y") & (df['made_it_to_contract'] == 1) & (df['plan_term'] != 1) & (df['plan_name'] != "RollOff Plan")},
    {"query_name":"EarlyPlanChangers","conditions":(df['is_sherpa'] != "y") & (df['made_it_to_contract'] !=1) & (df['plan_term'] != 1) & (df['plan_name'] != "RollOff Plan")},
    {"query_name":"MTMPlan","conditions":(df['is_sherpa'] != "y") & (df['plan_term'] == 1) & (df['plan_name'] != "RollOff Plan")},   
    {"query_name":"RollOff Plan","conditions":( (df['plan_term'] == 1) & ((df['plan_name'] == "RollOff Plan")))}   
]

### Generating Pivot Tables and Exporting it to Excel Document

In [ ]:
def create_and_export_pivot_tables(df,queries,output_excel_filename):
    workbook= Workbook()
    
    for query in queries:
            query_name = query['query_name']
            conditions = query['conditions']
            
            filtered_df = df[conditions]
            
            month = pd.pivot_table(filtered_df, values=['account number','moved_to_variable','customer_renewed','customer_left','rolled','plan energy charge','next plan energy charge'], index=['year','month'],aggfunc= {'account number':'count','moved_to_variable':np.sum,'customer_renewed':np.sum,'customer_left':np.sum,'rolled':np.sum,'plan energy charge':np.mean,'next plan energy charge':np.mean},margins=True,dropna=False)
            month = month.rename(columns = {'account number':'total'})
            month['overall renewed #'] = ((month['moved_to_variable'])+ month['customer_renewed'])
            month['% to Variable'] = ((month['moved_to_variable'] / month['total'])*100).apply(lambda x: round(x, 1))
            month['% Contract Renewed'] = ((month['customer_renewed'] / month['total'])*100).apply(lambda x: round(x, 1))
            month['% Overall Renewed'] = ((month['customer_renewed'] / month['total']) *100).apply(lambda x: round(x, 1)) + ((month['moved_to_variable'] / month['total'])*100).apply(lambda x: round(x, 1))
            month['% Rolled'] = ((month['rolled'] / month['total'])*100 ).apply(lambda x: round(x, 1))
            month['% Left'] = ((month['customer_left'] / month['total'])*100 ).apply(lambda x: round(x, 1))
            month['Energy_Charge_Change'] = ((month['next plan energy charge'] - month['plan energy charge']))
            cols = ['total','moved_to_variable','customer_renewed','overall renewed #','rolled','customer_left','% Contract Renewed','% to Variable','% Rolled','% Overall Renewed','% Left','plan energy charge','next plan energy charge','Energy_Charge_Change']
            month = month[cols]
            month = month.dropna()
            
            year = pd.pivot_table(filtered_df, values=['account number','moved_to_variable','customer_renewed','customer_left','rolled','plan energy charge','next plan energy charge'], index=['year'],aggfunc= {'account number':'count','moved_to_variable':np.sum,'customer_renewed':np.sum,'customer_left':np.sum,'rolled':np.sum,'plan energy charge':np.mean,'next plan energy charge':np.mean},margins=True,dropna=False)
            year = year.rename(columns = {'account number':'total'})
            year['overall renewed #'] = ((year['moved_to_variable'])+ year['customer_renewed'])
            year['% to Variable'] = ((year['moved_to_variable'] / year['total'])*100).apply(lambda x: round(x, 1))
            year['% Rolled'] = ((year['rolled'] / year['total'])*100).apply(lambda x: round(x, 1))
            year['% Contract Renewed'] = ((year['customer_renewed'] / year['total'])*100).apply(lambda x: round(x, 1))
            year['% Overall Renewed'] = ((year['customer_renewed'] / year['total']) *100).apply(lambda x: round(x, 1)) + ((year['moved_to_variable']/year['total'])*100).apply(lambda x: round(x, 1))
            year['% Left'] = ((year['customer_left'] / year['total'])*100).apply(lambda x: round(x, 1))
            year['Energy_Charge_Change'] = ((year['next plan energy charge'] - year['plan energy charge']))
            cols = ['total','moved_to_variable','customer_renewed','overall renewed #','rolled','customer_left','% Contract Renewed','% to Variable','% Rolled','% Overall Renewed','% Left','plan energy charge','next plan energy charge','Energy_Charge_Change']
            year = year[cols]
            year = year.dropna()

            sheet = workbook.create_sheet(title=query_name)
            
            for row_data in dataframe_to_rows(year,index=True,header=True): 
                sheet.append(row_data)
                
            for _ in range (4):
                sheet.append([])
                
            for row_data in dataframe_to_rows(month,index=True,header=True): 
                sheet.append(row_data)
            
    workbook.remove(workbook.active)
    workbook.save(output_excel_filename)
output_excel_filename = f'Renewals Report From may2022 to {date}.xlsx'
create_and_export_pivot_tables(df, queries,output_excel_filename)        
            